## Visualizing Spatial Data
The Spatially Enabled Dataframe has a `plot()` method that uses a syntax and symbology similar to [`matplotlib`](https://matplotlib.org) for visualizing features on a map. With this functionality, you can easily visualize aspects of your data both on a map and on a matplotlib chart using the same symbology!

Some unique characteristics of working with the visualization capabalities on the SDF:
- Uses Pythonic syntax
- Uses the same syntax as visualizing charts on Pandas DataFrames
- Uses symbology familiar to users of matplotlib
- Works on features and attributes simultaneously, eliminating to a great extent the need to iterate over all features (rows)
- Handles reading and writing to multiple formats aiding data conversion

Checkout the [Introduction to Spatially Enabled DataFrame](/python/guide/features-module/introduction-to-spatial-dataframe) guide to learn how to create a Spatial DataFrame.

In this guide, you will learn about:

 * [Quickstart](#Quickstart)
   * [Plotting the `DataFarme`](#Plotting-the-SpatialDataFrame)
 * [Understanding renderers](#Understanding-renderers)
   * [Supported renderers](#Supported-renderers)
   * [Visualizing unique values](#Visualizing-unique-values)
   * [Visualizing classes with different colors](#Visualizing-classes-with-different-colors)
 * [ColorMaps and Colors](#Colormaps-and-Colors)
 * [Symbology for Simple Renderers](#Symbology-for-Simple-Renderers)
   * [Getting the different symbol styles](#Getting-the-different-symbol-styles)
   * [Visualizing line features suing simple renderers](#Visualizing-line-features-using-simple-symbols)
   * [Visualizing area features using simple renderers](#Visualizing-area-features-using-different-symbols)

## Quickstart
Let us read a census data on major cities and load that into a `Spatially Enabled DataFrame`

In [4]:
from arcgis import GIS
gis = GIS()
# create an anonymous connection to ArcGIS Online and get a public item
item = gis.content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")
flayer = item.layers[0]

# Specify a SQL query and get a sub-set of the original data as a DataFrame
df = flayer.query(where="AGE_45_54 < 1500").sdf

# Visualize the top 5 records
df.head()

,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_5_9,AGE_65_74,AGE_75_84,...,PLACEFIPS,POP2010,POPULATION,POP_CLASS,RENTER_OCC,SHAPE,ST,STFIPS,VACANT,WHITE
0,1413,1381,1106,2138,1815,1411,979,1557,525,307,...,0468080,14287,14980,6,1074,null,AZ,04,261,9196
1,727,738,677,1380,1185,1333,1087,740,661,444,...,0602042,9932,10239,6,2056,null,CA,06,267,8273
2,593,511,2323,2767,746,127,34,1229,4,2,...,0610561,10616,11869,6,2558,null,CA,06,296,7530
3,888,988,900,1729,1479,1443,959,766,514,280,...,0613560,10866,11195,6,761,null,CA,06,86,5898
4,1086,1228,1013,1822,1759,1478,1112,925,687,477,...,0614974,12823,13009,6,1763,null,CA,06,88,6930


> Note: If the above cell looks new to you, please checkout the [Introduction to Spatially Enabled DataFrame](/python/guide/features-module/introduction-to-spatial-dataframe) guide.

Let us create a map of the United States

In [2]:
m1 = GIS().map('United States')
m1

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

![map of US diamonds](http://esri.github.io/arcgis-python-api/notebooks/nbimages/13_3_map_dfplot1a.png)

In [3]:
m1.zoom = 4
m1.center = [39,-98]

### Plotting the DataFrame
You can quickly visualize the points by calling the `plot()` method off the dataframes `spatial` accessor and passing the map you created above.

In [4]:
df.spatial.plot(map_widget= m1)

True

You can customize the symbols, their color, shape, border etc. like you would in a typical matplotlib plot. The rest of this guide talks about such customizations and suggestions to visualize your spatial and non-spatial data.

The code below plots the same set of points on a new map using a common structure used amongst many different Python packages for defining symbology. It is built off of the matplotlib libraries for simple, straightforward plotting. We'll explain some of the parameters below, and the [`plot()`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.SpatialDataFrame.plot) API Reference outlines more options.

In [5]:
m2= GIS().map('United States', zoomlevel=4)
m2

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

![map of US diamonds](http://esri.github.io/arcgis-python-api/notebooks/nbimages/13_3_map_dfplot1.png)

In [6]:
m2.center=[39,-98]
df.spatial.plot(map_widget=m2,
            symbol_type='simple',
            symbol_style='d', # d - for diamonds
            colors='Reds_r',
            cstep=10,
            outline_color='Blues',
            marker_size=10)

True

If this type of plotting is new to you, this is a good time to checkout [`matplotlib.pyplot`](https://matplotlib.org/api/pyplot_summary.html) documentation and Pandas dataframe [`plot`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) documentation to get an idea.

## Understanding renderers

In the snippets above, you visualized all the features with the same symbology. While this helps understand the geographic distribution of your data, your `Spatially Enabled DataFrame` object can do much more. For instance, you can group the features based on certain attributes that you choose and symbolize each group by a certain color or size or both. To help with this, let us understand the concept of **renderers**.

[`Renderers`](https://developers.arcgis.com/documentation/common-data-types/renderer-objects.htm) define how to visually represent a `feature layer` by defining [`symbols`](https://developers.arcgis.com/documentation/common-data-types/symbol-objects.htm) to represent individual features. The `SeDF` provides you with functionality to control the way features appear by choosing the `symbol` the renderer uses.

Previous versions of the ArcGIS API for Python provided a method to specify a renderer manually, but you had to know details about the renderer before you drew your data. The [`map.add_layer()`]() method did not provide access to all options avialble for rendering datasets. The new visualization capabilities provided by the SDF allow you to draw spatial data quickly and easily with access to more rendering options.

### Supported renderers  

The renderering options below are documented in further detail in the [`DatFrame Reference`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html). At a high level, you have the following renderers:

| Renderer     	| Syntax 	| Explanation                                                                                 	|
|--------------	|--------	|---------------------------------------------------------------------------------------------	|
| Simple       	| 's'    	| renders using one symbol only (the examples in [quickstart](#quickstart) above)             	|
| Unique       	| 'u'    	| renders each unique value with a different symbol. Suitable for categorical columns         	|
| Unique       	| 'u-a'    	| renders each unique value with a different symbol using arcade expressions. Suitable for categorical columns         	|
| Class breaks 	| 'c'    	| renders each group of values with a different color or size. Suitable for numerical columns 	|
| Heatmap      	| 'h'    	| renders density of point data as raster of varying colors                                   	|

### Visualizing unique values
Let us explore the major cities census data further and explore if there are any categorical columns

In [7]:
df.columns

Index(['AGE_10_14', 'AGE_15_19', 'AGE_20_24', 'AGE_25_34', 'AGE_35_44',
       'AGE_45_54', 'AGE_55_64', 'AGE_5_9', 'AGE_65_74', 'AGE_75_84',
       'AGE_85_UP', 'AGE_UNDER5', 'AMERI_ES', 'ASIAN', 'AVE_FAM_SZ',
       'AVE_HH_SZ', 'BLACK', 'CAPITAL', 'CLASS', 'FAMILIES', 'FEMALES',
       'FHH_CHILD', 'FID', 'HAWN_PI', 'HISPANIC', 'HOUSEHOLDS', 'HSEHLD_1_F',
       'HSEHLD_1_M', 'HSE_UNITS', 'MALES', 'MARHH_CHD', 'MARHH_NO_C',
       'MED_AGE', 'MED_AGE_F', 'MED_AGE_M', 'MHH_CHILD', 'MULT_RACE', 'NAME',
       'OBJECTID', 'OTHER', 'OWNER_OCC', 'PLACEFIPS', 'POP2010', 'POPULATION',
       'POP_CLASS', 'RENTER_OCC', 'SHAPE', 'ST', 'STFIPS', 'VACANT', 'WHITE'],
      dtype='object')

The `ST` column contains state names in abbreviation is a good candidate. We could symbolize the points such that all which fall under the same state get the same symbol:

In [8]:
df[['ST', 'NAME']].head()

,ST,NAME
0,AZ,Somerton
1,CA,Anderson
2,CA,Camp Pendleton South
3,CA,Citrus
4,CA,Commerce


In [9]:
gis = GIS()
m3 = gis.map('Reno, NV', zoomlevel=4)
m3

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

![unique value renderer](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_sdf_unique_value_renderer_1.png)

In [10]:
df.spatial.plot(map_widget = m3,
                renderer_type='u', # specify the unique value renderer using its notation 'u'
                col='ST'  # column to get unique values from
               )

True

By default, the API picks a unique color from the [`jet`](https://matplotlib.org/tutorials/colors/colormaps.html) colormap for each category (US State names in this case) and assigns it. You can override this by choosing your own colormap (`prism` for instance) and customizing the symbol style, symbol type, outline color, thickness, transparency etc. Keep reading to learn more about these customizations.

It is a good cartographic practice to not have too many unique values in your legend. Human eyes can only preceive so many colors and there are only a limited number of colors to choose from in a given color map before the colors become indistinguishable.

As a safe practice, the Python API caps the maximum number of unique values to `256`, the remaining are generalized and plotted with a default color. If you have several unique values, you should consider picking a different column that better represents your data or consider using a different renderer.

### Visualizing Unique Values with Arcade Expressions

Arcade is an expression language that can be used across the ArcGIS Platform. Whether writing simple scripts to control how features are rendered, or expressions to control label text, Arcade provides a simple scripting syntax to deliver these capabilities.

In the sense of visualization, Arcade expressions are used to create rich and dynamic symbology.  This example will follow the [JavaScript guide](https://developers.arcgis.com/javascript/latest/guide/arcade/index.html#visualization).

#### Obtain the Data

Access the enterprise to gain access to the FeatureLayer information and query it into a `Spatially Enabled DataFrame`.

In [1]:
from arcgis.gis import GIS
gis = GIS()

In [ ]:
item = gis.content.get("8444e275037549c1acab02d2626daaee")
flayer = item.layers[0]
df = flayer.query().sdf

In [9]:
fs = flayer.query()


In [18]:
from arcgis.geometry import Geometry
g = Geometry(fs.features[0].geometry)
g.as_arcpy


#### Write out the Arcade Expressions and Stops

Arcade expressions require the stops to be manually provided.  In this case, we first create an opacity visual variable based on a percent of dominate parties in registered citizens.

In [6]:
opacity_expression = ("var republican = $feature.MP06025a_B;var democrat = $feature.MP06024a_B;"
                      "var independent = $feature.MP06026a_B;var parties = [republican, democrat, independent];"
                      "var total = Sum(parties);var max = Max(parties);return (max / total) * 100;")
opacity_stops = [
    { "value": 33, "transparency": 0.05 * 255, "label": "< 33%" },
    { "value": 44, "transparency": 1.0 * 255, "label": "> 44%" }]

Next we develop another Arcade expression to obtain the majority party in a given county.

In [7]:
arcade_expression = ("var republican = $feature.MP06025a_B;var democrat = $feature.MP06024a_B;"
                     "var independent = $feature.MP06026a_B;var parties = [republican, democrat, independent];"
                     "return Decode( Max(parties),republican, 'republican', democrat, 'democrat',independent, "
                     "'independent','n/a' );")
uv = [{"label":"Democrat","symbol":{"type":"esriSFS","color":[0,195,255,255],
                                    "outline":{"type":"esriSLS","color":[0,0,0,51],
                                               "width":0.5,"style":"esriSLSSolid"},
                                    "style":"esriSFSSolid"},"value":"democrat"},
      {"label":"Republican","symbol":{"type":"esriSFS","color":[255,0,46,255],"outline":{
          "type":"esriSLS","color":[0,0,0,51],"width":0.5,"style":"esriSLSSolid"},
                                      "style":"esriSFSSolid"},"value":"republican"},
      {"label":"Independent/other party","symbol":{"type":"esriSFS","color":[250,255,0,255],
                                                   "outline":{"type":"esriSLS","color":[0,0,0,51],
                                                              "width":0.5,"style":"esriSLSSolid"},
                                                   "style":"esriSFSSolid"},"value":"independent"}]

In [18]:
gis = GIS()
m3_ua = gis.map('United States', zoomlevel=4)
m3_ua

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

#### Visualize the Data

Provide the color scheme, and the `arcade_expression` to render the data in a dynamic/rich form.

In [ ]:
df.spatial.plot(colors='coolwarm',
                map_widget=m3_ua,
                renderer_type='u-a',
                unique_values=uv,
                opacity_stops=opacity_stops,
                opacity_expression=opacity_expression,
                arcade_expression=arcade_expression)

In [22]:
m3_ua.extent

{'type': 'extent',
 'xmin': -16105138.185829269,
 'ymin': 2814150.199672489,
 'xmax': -6546229.176600808,
 'ymax': 6727726.047872473,
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}

### Visualizing classes with different colors
Often, you may want to classify the numerical values in your data into groups and visualize them on a map. You can accomplish this with a **class break renderer** which splits your data into specific number of groups and uses **color** to differentiate each group. You can choose the algorithm that performs the class splits or go with the defualt.

Let us visualize the same major cities point dataset using its `POPULATION` column.

In [ ]:
df[['ST', 'NAME', 'POPULATION']].head()

In [ ]:
m4 = gis.map('Reno, NV', zoomlevel=4)
m4

![class color renderer](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_sdf_class_breaks_color_1.png)

In [ ]:
df.spatial.plot(map_widget=m4,
               renderer_type='c',  # for class breaks renderer
               method='esriClassifyNaturalBreaks',  # classification algorithm
               class_count=20,  # choose the number of classes
               col='POPULATION',  # numeric column to classify
               cmap='prism',  # color map to pick colors from for each class
               alpha=0.7  # specify opacity
               )

You can take this further by plotting a histogram of the same `POPULATION` column and try to match the colors. To get the class breaks for the above map, you can inspect the layer definition from `MapView` object as shown below:

In [ ]:
m4.layers

In [ ]:
dict(m4.layers[0].layer.layerDefinition.drawingInfo.renderer).keys()

In [ ]:
class_breaks = m4.layers[0].layer.layerDefinition.drawingInfo.renderer.classBreakInfos
print(len(class_breaks))

We can loop through each of the class breaks and obtain the min, max values and the colors for each class.

In [ ]:
cbs_list = []
cmap_list = []
for cb in class_breaks:
    print(cb.description)  # print the class break labels
    cbs_list.append(cb.classMaxValue)
    cmap_list.append([x/255.0 for x in cb.symbol.color])

Now that we have the individual class splits and the colors, we can plot a histogram using the same breaks and colors.

In [ ]:
import matplotlib.pyplot as plt
# build a histogram for the same class breaks
n, bins, patches = plt.hist(df['POPULATION'], bins=cbs_list)

# apply the same color for each class to match the map
idx = 0
for c, p in zip(bins, patches):
    plt.setp(p, 'facecolor', cmap_list[idx])
    idx+=1

plt.title('Histogram of POPULATION column')

The shape and color of the bars in the histogram match with the number and color of major cities drawn on the map.

Thus using the `Spatially Enabled DataFrame` we can easily visualize both the spatial and numeric distribution of data using the same symbology.

## Colormaps and Colors
The examples above use the default colormap `jet`. However you can customize them by using the `colors` paramater. You can specify colors as:
 * a string representing [named colors](https://matplotlib.org/examples/color/named_colors.html)
 * an array of [RGB](http://www.tomjewett.com/colors/rgb.html) values
 * a named [color ramp](https://matplotlib.org/examples/color/colormaps_reference.html). 

#### Color Array

RGB + Alpha values can be used to create colors for symbols called in the `plot` method.  RGB stands for red, green, and blue respectively. Each RGB value is a value between 0-255, and the alpha value is a number between 0-255.

**Example to produce :**

    color = [255,0,100,1]

The above example produces a purplish color. Many websites provide details about using colors. For example, see [here](https://www.rapidtables.com/web/color/RGB_Color.html) for a color codes chart.

#### Color Maps

A color map is a collection of string values that can be given to generate a series of related colors from a defined set.

Color maps can be viewed here: https://matplotlib.org/examples/color/colormaps_reference.html

#### Color Map Helpers

To better understand the syntax for each input type, the ArcGIS API for Python provides some helper functions:

In [ ]:
from arcgis.mapping import display_colormaps

The **display_colormaps** function provides a quick, easy way to visualize the pre-defined set of colormaps you can use.  

In [ ]:
display_colormaps()

![colormap display](http://esri.github.io/arcgis-python-api/notebooks/nbimages/13_display_colormaps.png)

You can retrieve a list of colormaps as well:

In [ ]:
from arcgis.mapping import symbol

colormaps = symbol.ALLOWED_CMAPS
for a,b,c,d,e in zip(colormaps[::5], colormaps[1::5], colormaps[2::5], colormaps[3::5], colormaps[4::5]):
    print("{:<20}{:<20}{:<20}{:<20}{:<}".format(a,b,c,d,e))

You can enter a list of color ramp names as input to the `display_colormaps` function to filter the output:

In [ ]:
display_colormaps(['Greens_r', 'PRGn', 'Dark2', 'Set1'])

![selected color ramps](http://esri.github.io/arcgis-python-api/notebooks/nbimages/13_selected_colormaps.png)

The examples above used `jet` which is the default and some used `prism` both of which can be seen in the list above. You can customize your charts by using a different colormap of your choice.

## Symbology for Simple Renderers

The ArcGIS API for Python provides you the ability to set symbol types so you control data appearance. The  [`show_styles`]() function in the `arcgis.mapping` module assists you with the syntax to define symbols.

#### Getting the different symbol styles

In [ ]:
from arcgis.mapping import show_styles

In [ ]:
# get the styles that are relevant to the current geometry type (points)
show_styles(df.spatial.geometry_type[0])  # the DataFrame in this example is of Point type

In [ ]:
show_styles('Polygon')

In [ ]:
show_styles('Line')

For instance, you can represent the same set of points using square symbols instead of default circle using the snippet shown below:

In [ ]:
m7 = GIS().map('United States', 4)
m7

![map of US squares](http://esri.github.io/arcgis-python-api/notebooks/nbimages/13_4_map_dfplot2.png)

In [ ]:
m7.center = [39, -98]
df.spatial.plot(map_widget=m7,
        symbol_type='simple',
        symbol_style='s',  # for square
        cmap='Greens_r',   # color map is Greens_r for green reversed
        cstep=35,   # the individual color in the colormap
        outline_color='binary',
        marker_size=5,
        line_width=.5,)

### Visualizing line features using simple symbols
Let us search for USA freeway layer and visualize it by looping through the different line symbols

In [ ]:
search_result = gis.content.search('title:USA freeway system AND owner:esri', 
                                  item_type = 'Feature Layer')
freeway_item = search_result[0]
freeway_item

In [ ]:
freeway_sdf = freeway_item.layers[0].query().sdf
freeway_sdf.head()

In [ ]:
m8 = gis.map('USA', 10)
m8

![](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_sdf_lines_animated_1.gif)

In [ ]:
m8.center=[34.05,-118.2]
m8.zoom=12
m8.basemap='dark-gray'

In [ ]:
import time
for row, style in show_styles('Line').iterrows():
    m8.remove_layers()
    time.sleep(1)  # sleep for 1 second so animation can finish
    print(style['ESRI_STYLE'] + " : " + style['MARKER'])
    freeway_sdf.spatial.plot(map_widget=m8,
                     cmap='Spectral',  # user a different color map
                     symbol_type='simple',
                     symbol_style=style['MARKER']
                    )

### Visualizing area features using different symbols

In [ ]:
from arcgis.features import FeatureLayer
fl = FeatureLayer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/Census/MapServer/2")
county_sdf = fl.query("STATE_NAME='Washington'", out_sr=4326).sdf
county_sdf.head()

In [ ]:
m9 = gis.map('Seattle, WA', zoomlevel=6)
m9.basemap='gray'
m9

![](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_sdf_animate_polygons_1.gif)

In [ ]:
for idx, style in show_styles('Polygon').iterrows():
    m9.remove_layers()
    time.sleep(0.5)
    
    print(style['ESRI_STYLE'] + " : " + style['MARKER'])
    county_sdf.spatial.plot(map_widget=m9,
                           cmap = 'RdPu',  # use a red to purple color map
                           symbol_type='simple',
                           symbol_style=style['MARKER'],
                           outline_style='s',
                           outline_color=[0,0,0,255],
                           line_width=1.0)

**Conclusion**


The `Spatially Enabled DataFrame` gives you powerful visualization capabilities that allows you to plot your data on the interactive map widget. You specify colors and symbols using the same syntax that you would specify for a normal Pandas or a matplotlib plot.